# ACT Dark Sector: Unified Dark Matter and Dark Energy
## From Causal Networks to Cosmological Observations

This notebook explores the **unified dark sector** in Algebraic Causality Theory, where dark matter and dark energy emerge naturally from topological defects and causal potential in the network.

**Key Concepts**:
- Topological defects as dark matter
- Causal potential as dark energy
- Memory-potential unification
- Experimental predictions

**Author**: ACT Collaboration  
**Date**: 2024  
**License**: MIT

In [ ]:
# Cell 1: Setup and Imports
# =========================
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from scipy import stats, optimize, integrate
from IPython.display import display, HTML, Markdown
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Configure plotting
rcParams['figure.figsize'] = (12, 8)
rcParams['font.size'] = 12
sns.set_style("whitegrid")
sns.set_palette("husl")

print("\u2705 Dark Sector Analysis imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

# \u1f30c ACT Dark Sector Theory

## Unified Dark Field \u03a8 = M + i\u03a6

In ACT, dark matter and dark energy are unified through a complex field:

$$
\Psi(x) = M(x) + i\Phi(x)
$$

Where:
- **M(x)**: Memory field (real part) - encodes causal history
- **\u03a6(x)**: Causal potential (imaginary part) - encodes future possibilities
- **Interaction**: $\lambda |\Psi|^4$ term couples memory and potential

### Emergence from Causal Networks:

1. **Dark Matter** originates from **topological defects** in causal structure
2. **Dark Energy** emerges from **causal potential** of future possibilities
3. **Unification** occurs through algebraic relations in the causal algebra

### Action for Dark Sector:

$$
S_{\text{dark}} = \int d^4x \left[ |\partial\Psi|^2 + m^2|\Psi|^2 + \lambda|\Psi|^4 + \xi R|\Psi|^2 \right]
$$

In [ ]:
# Cell 2: Import ACT Framework
# ============================
import sys
import os
sys.path.append('../src')

from act_model import AlgebraicCausalityTheory, ACTParameters
from act_dark_sector import DarkSectorExtension
from utils import ACTVisualizer

print("\u2705 ACT modules imported successfully!")

# Create or load ACT model
try:
    # Try to load from previous simulation
    model = AlgebraicCausalityTheory.load('act_model_demo.pkl')
    print("\u1f4c2 Loaded existing ACT model")
except:
    # Create new model
    print("\u1f504 Creating new ACT model...")
    params = ACTParameters(
        N=500,
        dimension=4,
        temperature=0.7,
        alpha=1.0,
        beta=0.1,
        gamma=0.01,
        seed=42
    )
    model = AlgebraicCausalityTheory(params)
    model.initialize_network(method='sprinkling')
    model.build_tetrahedral_complex(method='delaunay')
    model.build_causal_structure()
    model.thermalize(steps=30)
    model.calculate_observables()
    print("\u2705 New ACT model created")

print(f"Model info: {len(model.vertices)} vertices, {len(model.tetrahedra)} tetrahedra")

# \u1f50d Topological Defects as Dark Matter

## What are Topological Defects?

In causal networks, topological defects arise from:

1. **Monopoles**: Points with unusual causal connectivity
2. **Strings**: 1D filaments of high curvature
3. **Domain Walls**: 2D surfaces separating different causal phases

These defects naturally behave like **cold dark matter**:
- Gravitationally attractive
- Weakly interacting
- Form halos through hierarchical clustering

In [ ]:
# Cell 3: Detect Topological Defects
# ===================================

def detect_defects(model):
    """Detect topological defects in causal network."""
    if model.adjacency is None:
        return [], {}
    
    defects = []
    
    # Calculate vertex properties
    degrees = model.adjacency.sum(axis=1).A1
    mean_deg = np.mean(degrees)
    std_deg = np.std(degrees)
    
    # Get spatial coordinates
    coords = np.array([v.coordinates for v in model.vertices])
    spatial_coords = coords[:, 1:]  # Spatial components
    
    # 1. Monopoles (high degree vertices)
    monopole_threshold = mean_deg + 2 * std_deg
    monopoles = np.where(degrees > monopole_threshold)[0]
    
    for idx in monopoles:
        defects.append({
            'type': 'monopole',
            'vertex': int(idx),
            'degree': float(degrees[idx]),
            'mass_estimate': degrees[idx] / mean_deg,
            'position': coords[idx].tolist()
        })
    
    # 2. String endpoints (low degree vertices)
    string_threshold = mean_deg - 1.5 * std_deg
    strings = np.where(degrees < string_threshold)[0]
    
    for idx in strings:
        defects.append({
            'type': 'string_endpoint',
            'vertex': int(idx),
            'degree': float(degrees[idx]),
            'mass_estimate': 0.5,  # Lower mass
            'position': coords[idx].tolist()
        })
    
    # 3. Detect filaments (string-like structures)
    if len(spatial_coords) > 10:
        from scipy.spatial import KDTree
        tree = KDTree(spatial_coords)
        
        # Find vertices with anisotropic neighborhoods
        for i in range(len(model.vertices)):
            # Get neighbors within radius
            neighbors = tree.query_ball_point(spatial_coords[i], r=0.3)
            if len(neighbors) > 3:
                # Check if neighbors are aligned
                neighbor_vecs = spatial_coords[neighbors] - spatial_coords[i]
                if len(neighbor_vecs) > 2:
                    # Calculate anisotropy
                    cov = np.cov(neighbor_vecs.T)
                    eigenvalues = np.linalg.eigvalsh(cov)
                    anisotropy = (eigenvalues[-1] - eigenvalues[0]) / np.sum(eigenvalues)
                    
                    if anisotropy > 0.7:  # Highly anisotropic -> filament
                        defects.append({
                            'type': 'filament_point',
                            'vertex': i,
                            'anisotropy': float(anisotropy),
                            'position': coords[i].tolist()
                        })
    
    # Classify defects
    defect_counts = {}
    for defect in defects:
        defect_type = defect['type']
        defect_counts[defect_type] = defect_counts.get(defect_type, 0) + 1
    
    return defects, defect_counts

# Run defect detection
defects, defect_counts = detect_defects(model)

# Display results
display(Markdown(f"""
## \u1f52c Topological Defects Detected:

**Total defects**: {len(defects)}

**Breakdown by type**:
"""))

for defect_type, count in defect_counts.items():
    display(Markdown(f"- **{defect_type}**: {count} defects"))

# Calculate dark matter fraction
total_vertices = len(model.vertices)
dm_fraction = len(defects) / total_vertices

display(Markdown(f"""
**Dark matter fraction**: {dm_fraction:.3f}

**Comparison with observations**:
- Observed \u03a9_dm \u2248 0.268
- Our estimate: \u03a9_dm \u2248 {dm_fraction:.3f}
- **Scaling factor needed**: {0.268/dm_fraction:.1f}
"""))

In [ ]:
# Cell 4: Visualize Defects in 3D
# ================================

# Extract coordinates
coords = np.array([v.coordinates for v in model.vertices])

# Create color coding for defects
vertex_colors = np.array(['lightgray'] * len(model.vertices))
defect_positions = []
defect_types = []

for defect in defects:
    idx = defect['vertex']
    if defect['type'] == 'monopole':
        vertex_colors[idx] = 'red'
    elif defect['type'] == 'string_endpoint':
        vertex_colors[idx] = 'blue'
    elif defect['type'] == 'filament_point':
        vertex_colors[idx] = 'green'
    
    defect_positions.append(coords[idx])
    defect_types.append(defect['type'])

# Create 3D plot
fig = plt.figure(figsize=(14, 10))

# Plot 1: Spatial distribution with defects
ax1 = fig.add_subplot(221, projection='3d')

# Plot all vertices
scatter_all = ax1.scatter(coords[:, 1], coords[:, 2], coords[:, 3], 
                         c='lightgray', s=10, alpha=0.3, label='Regular vertices')

# Plot defects
if defect_positions:
    defect_array = np.array(defect_positions)
    colors = []
    for d_type in defect_types:
        if d_type == 'monopole':
            colors.append('red')
        elif d_type == 'string_endpoint':
            colors.append('blue')
        else:
            colors.append('green')
    
    scatter_defects = ax1.scatter(defect_array[:, 1], defect_array[:, 2], defect_array[:, 3],
                                 c=colors, s=50, alpha=0.8, label='Defects')

ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Topological Defects in Causal Network')
ax1.legend()

# Plot 2: Defect type distribution
ax2 = fig.add_subplot(222)
types, counts = zip(*defect_counts.items())
bars = ax2.bar(range(len(types)), counts, color=['red', 'blue', 'green'][:len(types)])
ax2.set_xticks(range(len(types)))
ax2.set_xticklabels(types, rotation=45)
ax2.set_ylabel('Count')
ax2.set_title('Defect Type Distribution')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
            f'{int(height)}', ha='center', va='bottom')

# Plot 3: Mass distribution of monopoles
ax3 = fig.add_subplot(223)
monopole_masses = [d['mass_estimate'] for d in defects if d['type'] == 'monopole']
if monopole_masses:
    ax3.hist(monopole_masses, bins=15, alpha=0.7, color='red', edgecolor='black')
    ax3.set_xlabel('Mass (relative to average)')
    ax3.set_ylabel('Count')
    ax3.set_title('Monopole Mass Distribution')
    ax3.grid(True, alpha=0.3)
    
    # Fit power law
    masses_sorted = np.sort(monopole_masses)
    cdf = np.arange(1, len(masses_sorted)+1) / len(masses_sorted)
    ax3.loglog(masses_sorted, 1-cdf, 'ro', alpha=0.5, label='Empirical CDF')
    
    # Power law fit: P(>m) \u221d m^{-\u03b1}
    try:
        popt, _ = optimize.curve_fit(lambda x, alpha: x**(-alpha), 
                                    masses_sorted[masses_sorted > 1], 
                                    1-cdf[masses_sorted > 1])
        alpha_fit = popt[0]
        ax3.loglog(masses_sorted, masses_sorted**(-alpha_fit), 
                  'k--', label=f'Power law \u03b1={alpha_fit:.2f}')
        ax3.legend()
    except:
        pass

# Plot 4: Spatial clustering of defects
ax4 = fig.add_subplot(224)
if defect_positions:
    from scipy.spatial import distance
    
    # Calculate pairwise distances
    defect_array = np.array(defect_positions)[:, 1:]  # Spatial only
    if len(defect_array) > 2:
        distances = distance.pdist(defect_array)
        
        # Two-point correlation function
        bins = np.logspace(-2, 1, 20)
        hist, bin_edges = np.histogram(distances, bins=bins)
        
        # Normalize by volume
        volumes = 4/3 * np.pi * (bin_edges[1:]**3 - bin_edges[:-1]**3)
        density = len(defect_array) / np.prod(np.ptp(defect_array, axis=0))
        expected = density * volumes * len(defect_array) * (len(defect_array)-1) / 2
        
        # Correlation function \u03be(r) = (N_obs/N_exp) - 1
        with np.errstate(divide='ignore', invalid='ignore'):
            xi = np.where(expected > 0, hist/expected - 1, 0)
        
        ax4.plot(0.5*(bin_edges[1:] + bin_edges[:-1]), xi, 'o-', linewidth=2)
        ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        ax4.set_xlabel('Distance r')
        ax4.set_ylabel('\u03be(r)')
        ax4.set_title('Defect Two-Point Correlation Function')
        ax4.set_xscale('log')
        ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# \u26a1 Causal Potential as Dark Energy

## The Causal Potential \u03a6(x)

Dark energy in ACT emerges from the **causal potential** \u03a6(x), which measures:

$$
\Phi(x) = \sum_{y \in \text{spacelike}} \frac{1}{\tau(x,y)^2}
$$

Where \u03c4(x,y) is the proper time separation. This potential:

1. **Repulsive** at large scales -> cosmic acceleration
2. **Scale-dependent** -> evolving equation of state
3. **Unified** with memory field through \u03a8 = M + i\u03a6

In [ ]:
# Cell 5: Initialize and Analyze Dark Sector
# ==========================================

# Initialize dark sector
dark_sector = DarkSectorExtension(model)
dark_sector.initialize_unified_field()

# Calculate dark sector properties
if model.observables and 'dark_sector' in model.observables:
    dm = model.observables['dark_sector']['dark_matter']
    de = model.observables['dark_sector']['dark_energy']
    
    # Extract unified field
    unified_field = dark_sector.unified_field
    memory_field = np.real(unified_field)
    potential_field = np.imag(unified_field)
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    
    # Plot 1: Memory field distribution
    ax1 = axes[0, 0]
    ax1.hist(memory_field, bins=30, alpha=0.7, color='blue', density=True)
    ax1.set_xlabel('Memory Field M(x)')
    ax1.set_ylabel('Probability Density')
    ax1.set_title('Memory Field Distribution')
    ax1.grid(True, alpha=0.3)
    
    # Fit exponential distribution
    try:
        loc, scale = stats.expon.fit(memory_field - np.min(memory_field) + 1e-10)
        x = np.linspace(memory_field.min(), memory_field.max(), 100)
        pdf = stats.expon.pdf(x - np.min(memory_field) + 1e-10, loc, scale)
        ax1.plot(x, pdf, 'r-', linewidth=2, label='Exponential fit')
        ax1.legend()
    except:
        pass
    
    # Plot 2: Potential field distribution
    ax2 = axes[0, 1]
    ax2.hist(potential_field, bins=30, alpha=0.7, color='green', density=True)
    ax2.set_xlabel('Potential Field \u03a6(x)')
    ax2.set_ylabel('Probability Density')
    ax2.set_title('Potential Field Distribution')
    ax2.grid(True, alpha=0.3)
    
    # Fit power law
    try:
        # Use logarithmic bins for power law
        positive_potential = potential_field[potential_field > 0]
        if len(positive_potential) > 10:
            hist, bin_edges = np.histogram(np.log10(positive_potential), bins=20)
            bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
            ax2.semilogy(10**bin_centers, hist/len(potential_field), 'ro', alpha=0.5)
    except:
        pass
    
    # Plot 3: Field correlation
    ax3 = axes[0, 2]
    ax3.scatter(memory_field, potential_field, alpha=0.5, s=10)
    ax3.set_xlabel('Memory Field M(x)')
    ax3.set_ylabel('Potential Field \u03a6(x)')
    ax3.set_title('Memory-Potential Correlation')
    ax3.grid(True, alpha=0.3)
    
    # Add linear fit
    try:
        coeffs = np.polyfit(memory_field, potential_field, 1)
        x_fit = np.array([memory_field.min(), memory_field.max()])
        y_fit = np.polyval(coeffs, x_fit)
        ax3.plot(x_fit, y_fit, 'r-', linewidth=2, 
                label=f'Slope = {coeffs[0]:.3f}')
        ax3.legend()
    except:
        pass
    
    # Plot 4: Field magnitude distribution
    ax4 = axes[1, 0]
    field_magnitude = np.abs(unified_field)
    ax4.hist(field_magnitude, bins=30, alpha=0.7, color='purple', density=True)
    ax4.set_xlabel('|\u03a8(x)| = \u221a(M\u00b2 + \u03a6\u00b2)')
    ax4.set_ylabel('Probability Density')
    ax4.set_title('Unified Field Magnitude')
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: Field phase distribution
    ax5 = axes[1, 1]
    field_phase = np.angle(unified_field)
    ax5.hist(field_phase, bins=30, alpha=0.7, color='orange', density=True)
    ax5.set_xlabel('arg(\u03a8(x)) = arctan(\u03a6/M)')
    ax5.set_ylabel('Probability Density')
    ax5.set_title('Unified Field Phase')
    ax5.grid(True, alpha=0.3)
    
    # Plot 6: Interaction energy density
    ax6 = axes[1, 2]
    interaction_density = np.abs(unified_field)**4
    ax6.hist(interaction_density, bins=30, alpha=0.7, color='brown', density=True)
    ax6.set_xlabel('|\u03a8|\u2074')
    ax6.set_ylabel('Probability Density')
    ax6.set_title('Interaction Energy Density')
    ax6.set_xscale('log')
    ax6.set_yscale('log')
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate interaction energy
    interaction_energy = dark_sector.calculate_interaction_energy()
    
    display(Markdown(f"""
    ## \u1f4ca Unified Dark Field Statistics:
    
    **Memory Field (M)**:
    - Mean: {np.mean(memory_field):.3e}
    - Std: {np.std(memory_field):.3e}
    - Skewness: {stats.skew(memory_field):.3f}
    
    **Potential Field (\u03a6)**:
    - Mean: {np.mean(potential_field):.3e}
    - Std: {np.std(potential_field):.3e}
    - Kurtosis: {stats.kurtosis(potential_field):.3f}
    
    **Unified Field \u03a8 = M + i\u03a6**:
    - Average magnitude: {np.mean(field_magnitude):.3e}
    - Interaction energy: {interaction_energy:.3e}
    - Memory-potential correlation: {np.corrcoef(memory_field, potential_field)[0,1]:.3f}
    
    **Physical Interpretation**:
    - Strong correlation -> unified dark sector
    - Exponential memory -> causal history decays
    - Power-law potential -> scale-free dark energy
    """))